In [2]:
import numpy as np
from scipy.linalg import expm
from matplotlib import pyplot as plt
import cirq

In [3]:
ident = np.array([[1., 0.], [0., 1.]])
sigma_x = np.array([[0., 1.], [1., 0.]])
sigma_y = np.array([[0., -1.0j], [1.0j, 0.]])
sigma_z = np.array([[1., 0.], [0., -1.]])

In [97]:
def omega():
    return np.random.rand() * np.pi * 2
def phi ():
    return np.random.rand() * np.pi * 2
def theta():
    return np.arccos(2 * np.random.rand() - 1)

def unitary_angles(n_qubits):
    omegas = [omega() for i in range(n_qubits)]
    phis = [phi() for i in range(n_qubits)]
    thetas = [theta() for i in range(n_qubits)]

    return [phis,omegas,thetas]

def unitary(phi,omega,theta):
    
    return expm(-0.5j * omega * sigma_z) @ expm(-0.5j * theta * sigma_x) @ expm(-0.5j * phi * sigma_z)

In [117]:
qubits = cirq.LineQubit.range(3)

def shadows(angles):

    shadows = cirq.Circuit()

# Preparing GHZ state

    shadows.append([cirq.H(qubits[0]),
                    cirq.CNOT(qubits[0],qubits[1]),
                    cirq.CNOT(qubits[1],qubits[2])
                    ])

# Unitaries

    shadows.append([cirq.rz(angles[0][0])(qubits[0]), cirq.rz(angles[0][1])(qubits[1]), cirq.rz(angles[0][2])(qubits[2]),

                    cirq.rx(angles[2][0])(qubits[0]), cirq.rx(angles[2][1])(qubits[1]), cirq.rx(angles[2][2])(qubits[2]),

                    cirq.rz(angles[1][0])(qubits[0]), cirq.rz(angles[1][1])(qubits[1]), cirq.rz(angles[1][2])(qubits[2])],
                    strategy=cirq.circuits.InsertStrategy.NEW_THEN_INLINE)

# Measuring

    shadows.append(cirq.measure(qubits[0],qubits[1],qubits[2],key='out'))

    return shadows

print(shadows(unitary_angles(3)))


0: ───H───@───────Rz(0.097π)───Rx(0.234π)───Rz(1.32π)───M('out')───
          │                                             │
1: ───────X───@───Rz(0.602π)───Rx(0.782π)───Rz(1.6π)────M──────────
              │                                         │
2: ───────────X───Rz(0.63π)────Rx(0.413π)───Rz(0.49π)───M──────────


In [125]:
projectors = np.zeros([2,2,2])
projectors[0,0,0] = 1
projectors[1,1,1] = 1

def snapshot(angles,results):
    
    rho = np.eye(1)
    for i_mmt, mmt in enumerate(results):
        U = unitary(angles[0][i_mmt],angles[1][i_mmt],angles[2][i_mmt])
        rho = np.kron(rho, 3 * np.conj((U.T)) @ projectors[:,:,mmt] @ U - np.eye(2))
    
    return rho

N_rep = 10000
rho = np.zeros([8,8],dtype='complex')
for repetitions in range(N_rep):
    angles = unitary_angles(3)
    circuit = shadows(angles)
    simulator = cirq.Simulator()
    result = simulator.run(circuit).measurements['out'][0]

    rho += snapshot(angles,result)

rho /= N_rep

rho_ex = np.zeros([8,8])
rho_ex[0,0] = 1/2
rho_ex[7,7] = 1/2
rho_ex[0,7] = 1/2
rho_ex[7,0] = 1/2
print(np.trace(np.conj((rho-rho_ex).T)@(rho - rho_ex)))







(0.012741618068250544+0j)


In [123]:
np.conj((rho-rho_ex).T)@(rho - rho_ex)

array([[ 1.71068030e-02+0.00000000e+00j,  1.25784298e-03+2.63527824e-03j,
        -4.52188821e-03+1.25899527e-03j, -4.93717891e-03+2.38595912e-03j,
        -3.84740421e-03-2.07002961e-03j,  4.03771247e-03+1.72993465e-03j,
        -8.01102510e-04+2.45740016e-03j,  3.80061964e-03+1.32616187e-03j],
       [ 1.25784298e-03-2.63527824e-03j,  8.00259694e-03+0.00000000e+00j,
         1.87703418e-03-1.12819727e-04j,  1.78776393e-03+2.22474883e-03j,
        -7.25110224e-04+3.72986783e-03j, -3.91273544e-04+2.92581281e-03j,
        -1.53840282e-03+1.23653040e-03j,  1.79616347e-03+2.29354268e-03j],
       [-4.52188821e-03-1.25899527e-03j,  1.87703418e-03+1.12819727e-04j,
         1.54426784e-02+0.00000000e+00j,  3.88321454e-03-1.88668069e-03j,
        -2.60996838e-03-1.92435808e-03j,  4.19491012e-04-1.25198673e-03j,
         4.49441605e-03+5.20424730e-03j,  2.19261948e-03-2.02914558e-03j],
       [-4.93717891e-03-2.38595912e-03j,  1.78776393e-03-2.22474883e-03j,
         3.88321454e-03+1.88668069e